In [1]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai')
qdrant_key = userdata.get('qdrant')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install llama-index-vector-stores-qdrant llama-index-readers-file llama-index-embeddings-fastembed
!pip install qdrant_client fastembed
!pip install llama-index-llms-openai
!pip install llama-index-extractors-entity
!pip install llama-index llama-index-readers-web
!pip install llama-index-embeddings-openai

!pip install html2text
!pip install  tenacity

In [59]:
from qdrant_client import QdrantClient

from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode
from llama_index.core import (
    GPTVectorStoreIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
    Settings,
#    ChatPromptTemplate,
    PromptTemplate,
)
from llama_index.core.callbacks import(
    CallbackManager, LlamaDebugHandler, CBEventType,
)
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
#from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

from tenacity import ( retry, stop_after_attempt,  wait_random_exponential,)  # for exponential backoff

In [60]:
qdrant_client = QdrantClient(
    url="https://c5c4eea7-3717-4878-a79e-25946e0c048d.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_key,
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='test'), CollectionDescription(name='MFdatabase')]


In [61]:
vector_store = QdrantVectorStore(client=qdrant_client, collection_name="MFdatabase")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [55]:
pipeline = IngestionPipeline(
    transformations=[
        TokenTextSplitter(separator="。 ", chunk_size=512, chunk_overlap=128),
        #SentenceSplitter(chunk_size=25, chunk_overlap=0),
        #TitleExtractor(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)

In [92]:
#古いデータを消す
!rm data/*.pdf
!rm data/*.txt

新しいDocumentをDataフォルダにいれIngestion　Pipelineを実行　Qdrant　DBに格納

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_pipeline(documents):
  pipeline.run(documents=documents)
  return

run_pipeline(documents)

In [ ]:
#pipeline.run(documents=documents)

格納したDBよりINDEXにダウンロード

In [86]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store,)
query_engine = index.as_query_engine()

In [87]:
response = query_engine.query("残留溶媒について教えてください。日本語で回答してください")
print(response)

残留溶媒の管理は、それぞれの溶媒の濃度限度値毎に管理されます。残留溶媒については、質量管理であり、製造中に生成される溶媒も対象となります。特定の溶媒がPDE値を超える場合、一変申請が必要であり、他の溶媒が軽微変更届出の対象であっても、一変申請が必要です。


In [20]:
# text_splitter = TokenTextSplitter(
#         separator="。 ", chunk_size=512, chunk_overlap=128)

# extractors = [
#         #TitleExtractor(nodes=1, llm=llm), #5
#         #QuestionsAnsweredExtractor(questions=1, llm=llm), #3
#         # EntityExtractor(prediction_threshold=0.5),
#         # SummaryExtractor(summaries=["prev", "self"], llm=llm),
#         # KeywordExtractor(keywords=10, llm=llm),
#         # CustomExtractor()
#     ]

# transformations = extractors + [text_splitter]